In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import calendar

from sklearn.model_selection import train_test_split
from scipy.stats import ranksums
from scipy.stats import pearsonr

from sklearn.linear_model import LogisticRegression,LinearRegression

In [2]:
sentiment=pd.read_csv('UMCSENT.csv')

sentiment['DATE'] = pd.to_datetime(sentiment['DATE'],format='%Y/%m/%d')

IOError: File UMCSENT.csv does not exist

In [ ]:
sentiment['weekday']=sentiment['DATE'].map(lambda x:x.weekday())
sentiment['previous_sunday'] = sentiment['DATE'] - sentiment['weekday'].map(lambda x:timedelta(days=x+1))
sentiment.head()

In [ ]:
rolling_orders = pd.read_csv('rolling_window_ts.csv')

In [ ]:
rolling_orders = rolling_orders[['six','five','four','three','two','one','sunday_beginning']]
rolling_orders['sunday_beginning'] = pd.to_datetime(rolling_orders['sunday_beginning'])
#rolling_orders['monday_beginning'] = rolling_orders['sunday_beginning'] + timedelta(days=1)
rolling_orders.head()

In [ ]:
sentiment_ts_reg_ready = pd.merge(sentiment,rolling_orders,how='left',left_on='previous_sunday',right_on='sunday_beginning')

sentiment_ts_reg_ready.dropna(inplace=True)

sentiment_ts_reg_ready.head()

In [ ]:
sentiment_ts_reg_ready['UMCSENT_next_wk'] = sentiment_ts_reg_ready['UMCSENT'].shift(-1)
sentiment_ts_reg_ready = sentiment_ts_reg_ready.iloc[:-1,:]

sentiment_ts_reg_ready['UMCSENT'] = sentiment_ts_reg_ready['UMCSENT'].astype(float)
sentiment_ts_reg_ready['UMCSENT_next_wk'] = sentiment_ts_reg_ready['UMCSENT_next_wk'].astype(float)

sentiment_ts_reg_ready['UMCSENT_delta'] = sentiment_ts_reg_ready['UMCSENT_next_wk'] - sentiment_ts_reg_ready['UMCSENT']
sentiment_ts_reg_ready.head()

#### Modeling of sentiment_delta~one:six

In [ ]:
sentiment_ts_reg_ready['UMCSENT_direction'] = (sentiment_ts_reg_ready['UMCSENT_delta']>0).astype(int)

x = sentiment_ts_reg_ready[['six','five','four','three','two','one']]
y_bin = sentiment_ts_reg_ready['UMCSENT_direction']
y = sentiment_ts_reg_ready['UMCSENT_delta']

split = np.floor(0.75*sentiment_ts_reg_ready.shape[0]).astype(int)
print split

x_train = x.iloc[:split,:]
x_test = x.iloc[split:,:]
y_train = y_bin.iloc[:split]
y_train = np.ravel(y_train)
y_test = y_bin.iloc[split:]
y_test = np.ravel(y_test)

test_return = y.iloc[split:]
test_return = np.ravel(test_return)

In [ ]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(x_train,y_train)

In [ ]:
lr.coef_

In [ ]:
preds = lr.predict_proba(x_test)
preds = pd.DataFrame(preds)
preds = np.array(preds[1])
preds

In [ ]:
y_test = pd.DataFrame({'actual_binary':y_test,'actual_ret':test_return,'preds':preds})
y_test.sort_values(by='preds')
y_test.to_csv('testset.csv',index=False)

In [ ]:
print pearsonr(y_test['actual_ret'],y_test['preds'])

In [ ]:
print(np.mean(y_test[y_test['preds']>=0.5]['actual_binary']))

print(np.mean(y_test[y_test['preds']>=0.5]['actual_ret']))

In [ ]:
print(np.mean(y_test[y_test['preds']<=0.5]['actual_binary']))
      
print(np.mean(y_test[y_test['preds']<=0.5]['actual_ret']))

#### Regression Context

In [ ]:
sentiment_ts_reg_ready['UMCSENT_direction'] = (sentiment_ts_reg_ready['UMCSENT_delta']>0).astype(int)

x = sentiment_ts_reg_ready[['six','five','four','three','two','one']]
y_bin = sentiment_ts_reg_ready['UMCSENT_direction']
y = sentiment_ts_reg_ready['UMCSENT_delta']

split = np.floor(0.75*sentiment_ts_reg_ready.shape[0]).astype(int)
print split

x_train = x.iloc[:split,:]
x_test = x.iloc[split:,:]
y_train = y.iloc[:split]
y_test = y.iloc[split:]

In [ ]:
lr = LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
lr.coef_

In [ ]:
preds = lr.predict(x_test)
preds = pd.DataFrame(preds)
preds

In [ ]:
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True,drop=True)

In [ ]:
df = y_test.join(preds)
df.rename(columns={0:'preds'},inplace=True)
df

In [ ]:
print np.count_nonzero(df[df['preds']<0]['UMCSENT_delta'])
print np.median(df[df['preds']<0]['UMCSENT_delta'])

In [ ]:
print np.count_nonzero(df[df['preds']>0]['UMCSENT_delta'])
print np.median(df[df['preds']>0]['UMCSENT_delta'])

In [ ]:
pearsonr(df['UMCSENT_delta'],df['preds'])

In [ ]:
y_test1 = df.sort_values(by='preds')
y_test2 = y_test1.reset_index(drop=True)
y_test2